# Evaluating Movie Plot Generations via Rouge

In [0]:
!pip install rouge
from rouge import Rouge 
from datetime import datetime
from google.colab import files
import numpy as np
import pandas as pd
import time
import os
import nltk
from nltk import tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Utility

In [0]:
def find_plot_via_title(filename):
  title = filename[14:-4]
  plt = test_data.loc[test_data['Title'] == title]["Plot"].values[0]
  return plt

In [0]:
#sentence extraction: len = total len / 2
#sentence extraction: len = total len - 2
def extract_prediction_sent(text):
  sent_tokens = tokenize.sent_tokenize(text)
  size = max(len(sent_tokens) - 2, 1)
  beg_text = sent_tokens[size:]
  beg = " ".join(beg_text)
  return beg

def extract_prediction_word(text, amount):
  word_tokens = tokenize.word_tokenize(text)
  #size = max(int(len(word_tokens) * threshold), 1)
  prediction_tokens = word_tokens[amount:]
  prediction = " ".join(prediction_tokens)
  return prediction

def find_prefix(text, threshold):
  word_tokens = tokenize.word_tokenize(text)
  size = max(int(len(word_tokens) * threshold), 1)
  return size

In [0]:
def join_scores(s1, s2):
  temp = [{'rouge-1': {'f': 0, 'p': 0, 'r': 0},
          'rouge-2': {'f': 0, 'p': 0, 'r': 0},
          'rouge-l': {'f': 0, 'p': 0, 'r': 0}}]

  temp[0]['rouge-1']['f'] = s1[0]['rouge-1']['f'] + s2[0]['rouge-1']['f']
  temp[0]['rouge-1']['p'] = s1[0]['rouge-1']['p'] + s2[0]['rouge-1']['p'] 
  temp[0]['rouge-1']['r'] = s1[0]['rouge-1']['r'] + s2[0]['rouge-1']['r']  

  temp[0]['rouge-2']['f'] = s1[0]['rouge-2']['f'] + s2[0]['rouge-2']['f']
  temp[0]['rouge-2']['p'] = s1[0]['rouge-2']['p'] + s2[0]['rouge-2']['p'] 
  temp[0]['rouge-2']['r'] = s1[0]['rouge-2']['r'] + s2[0]['rouge-2']['r']  

  temp[0]['rouge-l']['f'] = s1[0]['rouge-l']['f'] + s2[0]['rouge-l']['f']
  temp[0]['rouge-l']['p'] = s1[0]['rouge-l']['p'] + s2[0]['rouge-l']['p'] 
  temp[0]['rouge-l']['r'] = s1[0]['rouge-l']['r'] + s2[0]['rouge-l']['r']  

  return temp

In [0]:
def average_scores(score, div):
  temp = [{'rouge-1': {'f': 0, 'p': 0, 'r': 0},
          'rouge-2': {'f': 0, 'p': 0, 'r': 0},
          'rouge-l': {'f': 0, 'p': 0, 'r': 0}}]

  temp[0]['rouge-1']['f'] = score[0]['rouge-1']['f'] / div
  temp[0]['rouge-1']['p'] = score[0]['rouge-1']['p'] / div
  temp[0]['rouge-1']['r'] = score[0]['rouge-1']['r']  / div

  temp[0]['rouge-2']['f'] = score[0]['rouge-2']['f'] / div
  temp[0]['rouge-2']['p'] = score[0]['rouge-2']['p'] / div
  temp[0]['rouge-2']['r'] = score[0]['rouge-2']['r']  / div 

  temp[0]['rouge-l']['f'] = score[0]['rouge-l']['f'] / div
  temp[0]['rouge-l']['p'] = score[0]['rouge-l']['p'] / div
  temp[0]['rouge-l']['r'] = score[0]['rouge-l']['r']  / div
  
  return temp

In [0]:
rouge = Rouge()

In [0]:
def rouge_calculations(directory_path, threshold, total):
  current_totals = [{'rouge-1': {'f': 0, 'p': 0, 'r': 0},
          'rouge-2': {'f': 0, 'p': 0, 'r': 0},
          'rouge-l': {'f': 0, 'p': 0, 'r': 0}}]

  current_prediction = [{'rouge-1': {'f': 0, 'p': 0, 'r': 0},
          'rouge-2': {'f': 0, 'p': 0, 'r': 0},
          'rouge-l': {'f': 0, 'p': 0, 'r': 0}}]
  directory = os.fsencode(directory_path)
  for file in os.listdir(directory):
    try:
      filename = os.fsdecode(file)
      full_path = os.path.join(directory_path, filename)
      file_open = open(full_path,mode='r')

      reference = find_plot_via_title(filename)
      num_tokens = find_prefix(reference, threshold)
      prediction_ref = extract_prediction_word(reference, num_tokens)

      plot = file_open.read()
      prediction_plot = extract_prediction_word(plot, num_tokens)
      
      total_scores = rouge.get_scores(plot, reference)
      prediction_scores = rouge.get_scores(prediction_plot, prediction_ref)
      
      current_totals = join_scores(current_totals, total_scores)
      current_prediction = join_scores(current_prediction, prediction_scores)
    except:
      total -= 1
  return average_scores(current_totals, total), average_scores(current_prediction, total)

In [0]:
def create_results_table(res25, res50, res75):
  labels = ["Rouge-1 F1", "Rouge-1 Precision", "Rouge-1 Recall", "Rouge-2 F1", "Rouge-2 Precision", "Rouge-2 Recall", "Rouge-L F1", "Rouge-L Precision", "Rouge-L Recall"]
  gen_results = pd.DataFrame(np.array([[res25['rouge-1']['f'],res50['rouge-1']['f'], res75['rouge-1']['f']], 
                                       [res25['rouge-1']['p'],res50['rouge-1']['p'], res75['rouge-1']['p']], 
                                       [res25['rouge-1']['r'],res50['rouge-1']['r'], res75['rouge-1']['r']],
                                       [res25['rouge-2']['f'],res50['rouge-2']['f'], res75['rouge-2']['f']], 
                                       [res25['rouge-2']['p'],res50['rouge-2']['p'], res75['rouge-2']['p']], 
                                       [res25['rouge-2']['r'],res50['rouge-2']['r'], res75['rouge-2']['r']],
                                       [res25['rouge-l']['f'],res50['rouge-l']['f'], res75['rouge-l']['f']], 
                                       [res25['rouge-l']['p'],res50['rouge-l']['p'], res75['rouge-l']['p']], 
                                       [res25['rouge-l']['r'],res50['rouge-l']['r'], res75['rouge-l']['r']]]),
                           columns = ["25% Prefix", "50% Prefix", "75% Prefix"],
                           index = labels)
  return gen_results

## Horror Genre Evaluation

In [0]:
#loading test file
test_data = pd.read_csv("/content/drive/My Drive/w266 final project/Genre_thrilhor_test.csv")
test_data = test_data.drop(['Unnamed: 0'], axis=1)
test_data.head()

,Title,Plot,Genre
0,Il Giorno prima,This story takes place when they experience th...,"['thriller', 'drama']"
1,The Ghost Inside,The Ghost Inside tells the story of a young mo...,"['movies', 'chinese', 'horror']"
2,Gigantor,The series is set in the year 2000. The show f...,"['anime', 'fiction', 'science', 'animation', '..."
3,Kimmy Dora and the Temple of Kiyeme,Kimmy and Dora are identical twins with diffe...,"['comedy', 'horror']"
4,Achamundu Achamundu,Senthil Kumar ([[Prasanna and Malini ([[Sneha...,"['thriller', 'drama']"


In [0]:
!unzip -q "/content/drive/My Drive/w266 final project/25_content_horror_0_300.zip"
!unzip -q "/content/drive/My Drive/w266 final project/25_content_horror_300_600.zip"

!unzip -q "/content/drive/My Drive/w266 final project/50_content_horror_0_100.zip"
!unzip -q "/content/drive/My Drive/w266 final project/50_content_horror_100_600.zip"

!unzip -q "/content/drive/My Drive/w266 final project/75_content_horror_0_260.zip"
!unzip -q "/content/drive/My Drive/w266 final project/75_content_horror_260_600.zip"

replace 25_content_horror/generated_test_plots/Plot Summary: Miami Blues.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace 25_content_horror/generated_test_plots/Plot Summary: A Killer Within.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace __MACOSX/50_content_horror/._generated_test_plots? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace 75_content_horror/generated_test_plots/Plot Summary: A Killer Within.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
# number of movie plots
path, dirs, files = next(os.walk("/content/25_content_horror/generated_test_plots"))
print("Number of 25% Files: {}".format(len(files)))

path, dirs, files = next(os.walk("/content/50_content_horror/generated_test_plots"))
print("Number of 50% Files: {}".format(len(files)))

path, dirs, files = next(os.walk("/content/75_content_horror/generated_test_plots"))
print("Number of 75% Files: {}".format(len(files)))

Number of 25% Files: 600
Number of 50% Files: 600
Number of 75% Files: 600


In [0]:
directory_25 = "/content/25_content_horror/generated_test_plots"
directory_50 = "/content/50_content_horror/generated_test_plots"
directory_75 = "/content/75_content_horror/generated_test_plots"

In [0]:
totals_25, pred_25 = rouge_calculations(directory_25, 0.25)
totals_50, pred_50 = rouge_calculations(directory_50, 0.5)
totals_75, pred_75 = rouge_calculations(directory_75, 0.75)

In [0]:
final_pred_results_horror = create_results_table(pred_25[0], pred_50[0], pred_75[0])
final_pred_results_horror

,25% Prefix,50% Prefix,75% Prefix
Rouge-1 F1,0.294754,0.284530,0.258799
Rouge-1 Precision,0.309974,0.305870,0.280232
Rouge-1 Recall,0.286698,0.277841,0.257494
Rouge-2 F1,0.044344,0.042093,0.039393
Rouge-2 Precision,0.047585,0.047061,0.042972
Rouge-2 Recall,0.042658,0.040487,0.038547
Rouge-L F1,0.212032,0.211676,0.202998
Rouge-L Precision,0.292537,0.273069,0.241011
Rouge-L Recall,0.177670,0.184249,0.187362


In [0]:
final_total_results_horror = create_results_table(totals_25[0], totals_50[0], totals_75[0])
final_total_results_horror

,25% Prefix,50% Prefix,75% Prefix
Rouge-1 F1,0.433776,0.592317,0.737350
Rouge-1 Precision,0.442954,0.592022,0.719553
Rouge-1 Recall,0.430086,0.597644,0.758694
Rouge-2 F1,0.239483,0.444427,0.634515
Rouge-2 Precision,0.245872,0.445228,0.619488
Rouge-2 Recall,0.236712,0.447866,0.652686
Rouge-L F1,0.413949,0.591498,0.744516
Rouge-L Precision,0.496172,0.642112,0.768088
Rouge-L Recall,0.364436,0.554261,0.725323


## Romance Genre Evaluation

In [0]:
#loading test file
test_data = pd.read_csv("/content/drive/My Drive/w266 final project/Genre_romance_test.csv")
test_data = test_data.drop(['Unnamed: 0'], axis=1)
test_data.head()

,Title,Plot,Genre
0,Cat And Mouse,Zhan Zhao is a court officer who learns of a ...,"['adventure', 'drama', 'movies', 'action', 'co..."
1,Rechipo,"Siva , a modern day Robinhood who steals money...",['romance']
2,My Chauffeur,Casey Meadows is a free-spirited young woman w...,"['indie', 'romance', 'romantic', 'comedy']"
3,Cannery Row,The story is about the skid row citizens of Mo...,"['americana', 'adaptation', 'drama', 'comedy',..."
4,Vigathakumaran,"Chandrakumar, son of a rich man in Trivandrum ...","['romance', 'drama', 'action']"


In [0]:
!unzip -q "/content/drive/My Drive/w266 final project/25_content_romance.zip"
!unzip -q "/content/drive/My Drive/w266 final project/50_content_romance.zip"
!unzip -q "/content/drive/My Drive/w266 final project/75_content_romance.zip"

In [0]:
# number of movie plots
path, dirs, files = next(os.walk("/content/25_content_romance/generated_test_plots"))
print("Number of 25% Files: {}".format(len(files)))

path, dirs, files = next(os.walk("/content/50_content_romance/generated_test_plots"))
print("Number of 50% Files: {}".format(len(files)))

path, dirs, files = next(os.walk("/content/75_content_romance/generated_test_plots"))
print("Number of 75% Files: {}".format(len(files)))

Number of 25% Files: 470
Number of 50% Files: 470
Number of 75% Files: 470


In [0]:
directory_25r = "/content/25_content_romance/generated_test_plots"
directory_50r = "/content/50_content_romance/generated_test_plots"
directory_75r = "/content/75_content_romance/generated_test_plots"

In [0]:
totals_25r, pred_25r = rouge_calculations(directory_25r, 0.25, 470)
totals_50r, pred_50r = rouge_calculations(directory_50r, 0.5, 470)
totals_75r, pred_75r = rouge_calculations(directory_75r, 0.75, 470)

In [0]:
final_pred_results_romance = create_results_table(pred_25r[0], pred_50r[0], pred_75r[0])
final_pred_results_romance

,25% Prefix,50% Prefix,75% Prefix
Rouge-1 F1,0.296406,0.290349,0.260697
Rouge-1 Precision,0.316662,0.308039,0.284046
Rouge-1 Recall,0.285422,0.282293,0.257988
Rouge-2 F1,0.044868,0.043864,0.038223
Rouge-2 Precision,0.049027,0.047521,0.044254
Rouge-2 Recall,0.042795,0.042336,0.037471
Rouge-L F1,0.214031,0.219227,0.209298
Rouge-L Precision,0.287209,0.273614,0.247245
Rouge-L Recall,0.180592,0.191641,0.194592


In [0]:
final_total_results_romance = create_results_table(totals_25r[0], totals_50r[0], totals_75r[0])
final_total_results_romance

,25% Prefix,50% Prefix,75% Prefix
Rouge-1 F1,0.435052,0.591658,0.735137
Rouge-1 Precision,0.449390,0.590413,0.717757
Rouge-1 Recall,0.427113,0.596665,0.756362
Rouge-2 F1,0.241475,0.441060,0.628860
Rouge-2 Precision,0.251113,0.440920,0.614459
Rouge-2 Recall,0.236274,0.444291,0.646656
Rouge-L F1,0.411584,0.586856,0.741490
Rouge-L Precision,0.487001,0.633370,0.762615
Rouge-L Recall,0.364627,0.551780,0.724156


In [0]:
final_pred_results_romance.to_csv()

',25% Prefix,50% Prefix,75% Prefix\nRouge-1 F1,0.23179833294116406,0.22695646972413003,0.2024185024115012\nRouge-1 Precision,0.24763897100605184,0.2407838745107601,0.22054749932519\nRouge-1 Recall,0.2232084184668564,0.22065939074439314,0.20031458166627736\nRouge-2 F1,0.03508788591452317,0.03428677569036663,0.02967846340239959\nRouge-2 Precision,0.03834080041327047,0.03714531257837778,0.03436130069145646\nRouge-2 Recall,0.03346667735716189,0.033092546396049895,0.02909421104852497\nRouge-L F1,0.1673786024766546,0.17136263282169953,0.16250959717592878\nRouge-L Precision,0.22460606063623795,0.21387485497694186,0.19197375264277025\nRouge-L Recall,0.14122813482909072,0.14979901547683624,0.1510908085930929\n'

In [0]:
final_pred_results_romance.to_json()

'{"25% Prefix":{"Rouge-1 F1":0.2317983329,"Rouge-1 Precision":0.247638971,"Rouge-1 Recall":0.2232084185,"Rouge-2 F1":0.0350878859,"Rouge-2 Precision":0.0383408004,"Rouge-2 Recall":0.0334666774,"Rouge-L F1":0.1673786025,"Rouge-L Precision":0.2246060606,"Rouge-L Recall":0.1412281348},"50% Prefix":{"Rouge-1 F1":0.2269564697,"Rouge-1 Precision":0.2407838745,"Rouge-1 Recall":0.2206593907,"Rouge-2 F1":0.0342867757,"Rouge-2 Precision":0.0371453126,"Rouge-2 Recall":0.0330925464,"Rouge-L F1":0.1713626328,"Rouge-L Precision":0.213874855,"Rouge-L Recall":0.1497990155},"75% Prefix":{"Rouge-1 F1":0.2024185024,"Rouge-1 Precision":0.2205474993,"Rouge-1 Recall":0.2003145817,"Rouge-2 F1":0.0296784634,"Rouge-2 Precision":0.0343613007,"Rouge-2 Recall":0.029094211,"Rouge-L F1":0.1625095972,"Rouge-L Precision":0.1919737526,"Rouge-L Recall":0.1510908086}}'